In [ ]:
import numpy as np
import pandas as pd
import random
import itertools
import time


In [ ]:
CAB_10_nodes_flow = pd.read_csv("10_nodes_CAB_flow.csv",delimiter= ";", header=None)                            


In [ ]:
CAB_10_nodes_cost = pd.read_csv("10_nodes_CAB_cost.csv",delimiter= ";", header=None)
for i in range(0, 10):
    CAB_10_nodes_cost[i] = [float(string.replace(',','.')) for string in CAB_10_nodes_cost[i] ]


In [ ]:
CAB_25_nodes_flow = pd.read_csv("25_nodes_CAB_flow.csv", header=None)                            


In [ ]:
CAB_25_nodes_cost = pd.read_csv("25_nodes_CAB_cost.csv", header=None)


In [ ]:
TR_25_nodes_flow = pd.read_csv("25_nodes_TR_flow.csv", header=None)


In [ ]:
TR_25_nodes_cost = pd.read_csv("25_nodes_TR_cost.csv", header=None)


In [ ]:
TR_55_nodes_flow = pd.read_csv("55_nodes_TR_flow.csv", header=None)


In [ ]:
TR_55_nodes_cost = pd.read_csv("55_nodes_TR_cost.csv", header=None)


In [ ]:
cab_10_capacity = [499513, 699318, 989036]
cab_25_capacity = [4270003, 5978004.2, 8454605.94]
tr_25_capacity = [13605802, 19048122.8, 26939487.96]
tr_55_capacity = [20376213.5, 28526698.9, 40344902.73]
hub_fixed_costs = [50000000, 100000000, 150000000]

In [ ]:
#RANDOM INITIAL SOLUTION GENERATOR
def random_initial_solution_generator(n_nodes, n_hubs):
    nodes = list(range(1, n_nodes+1))
    hubs = random.sample(nodes, n_hubs)
    solution = [0]*n_nodes
    
    #PLACING THE HUBS
    for hub in hubs:
        solution.pop(hub-1)
        solution.insert(hub-1, hub)
    
    spoke_index_list = [node-1 for node in nodes if node not in hubs]

    #PLACING THE SPOKES
    random.shuffle(spoke_index_list)

    for index in spoke_index_list:
        solution.pop(index)
        solution.insert(index, random.choice(hubs))

    return solution

In [ ]:
def least_cost_random_hub_based_solution_generator(n_nodes, n_hubs, cost_matrix):

    nodes = list(range(1, n_nodes+1))
    hubs = random.sample(nodes, n_hubs)
    solution = [0]*n_nodes
    for hub in hubs:
        nodes.remove(hub)
        spokes=nodes
    spoke_index_list = [(spoke-1) for spoke in spokes]
    hub_index_list = [(hub-1) for hub in hubs]



    for hub in hubs:
        solution.pop(hub-1)
        solution.insert(hub-1, hub)
    
    spoke_index_list = [node-1 for node in nodes if node not in hubs]

    
    #DECIDING THE HUBS OF THE NODES BASED ON THE LEAST COST
    spoke_hub_cost={}
    for spoke_index in spoke_index_list:
        spoke_hub_cost[spoke_index+1] = [(cost_matrix.iloc[spoke_index][hub-1],hub) for hub in hubs]
    spoke_hub_distribution = {}
    for spoke in list(spoke_hub_cost.keys()):
        costs = [ij[0] for ij in spoke_hub_cost[spoke]]
        hub = [ij[1] for ij in spoke_hub_cost[spoke] if ij[0] == min(costs)][0]
        spoke_hub_distribution[spoke] = hub  
        
    #CREATING SPOKE,HUB INDEX LIST
    spoke_hub_index_couples = [] 
    for spoke_index in spoke_index_list:
        for hub in hubs:
            if spoke_hub_distribution[spoke_index+1] == hub:
                spoke_hub_index_couples.append((spoke_index, hub-1))
    spoke_hubs_indexes = list(set(spoke_hub_index_couples))

    #PLACING THE SPOKES ACCORDING TO THE ABOVE LIST
    for index_couple in spoke_hubs_indexes:
        solution.pop(index_couple[0])
        solution.insert(index_couple[0],index_couple[1]+1 )

    return solution

In [ ]:
#NEIGHBOURHOOD STRUCTURE 1: swap the current random hub with random non-hub***1-s2.0-S0305054817302538
def NS1(solution, node):

    hub_to_spoke = random.choice(list(set(solution)))

    spokes = [node for node in range(1, node+1) if node not in set(solution)]

    spoke_to_hub = random.choice(spokes)   

    solution = [spoke_to_hub if x == hub_to_spoke else x for x in solution]
    solution[spoke_to_hub -1 ] = spoke_to_hub
    return solution
    
    

In [ ]:
#NEIGHBOURHOOD STRUCTURE 2: swap randomly chosen two hubs ***1-s2.0-S0305054817302538******
def NS2(solution, node):

    hub1_to_swap = random.choice(list(set(solution)))
    hubs_left = [hub for hub in set(solution) if hub != hub1_to_swap ]
    hub2_to_swap = random.choice(hubs_left)


    hub_spoke = {}
    for hub in list(set(solution)):
        hub_spoke[hub] = [i for i,val in enumerate(solution) if val==hub]
        for spokes in list(hub_spoke.values()):
            hub_spoke[hub] = [index for index in spokes if index != hub-1] #hubs and their spokes' indexes 

        
    hub1_index = hub_spoke[hub1_to_swap]
    hub2_index = hub_spoke[hub2_to_swap]
    
    for i in hub1_index:
        solution[i] = hub2_to_swap
    solution[(hub2_to_swap)-1] = hub2_to_swap
    for i in hub2_index:
        solution[i] = hub1_to_swap
    solution[(hub1_to_swap)-1] = hub1_to_swap
        

    return solution



In [ ]:
#NEIGHBOURHOOD STRUCTURE 3: Swapping the nodes of two hubs( random two hubs, random one node for each chosen hub)
def NS3(solution,node):

    hubs = list(set(solution))

    hubs_chosen = random.sample(hubs, 2)

    hub_spoke  = {}
    for hub in set(solution):
        hub_spoke[hub] = [i+1 for i,val in enumerate(solution) if val==hub]
        for spokes in list(hub_spoke.values()):
            hub_spoke[hub] = [index for index in spokes if index != hub-1] #hubs and their spokes' indexes 


    l1 = hub_spoke[hubs_chosen[0]]
    l2 = hub_spoke[hubs_chosen[1]]
    

    spoke1 = random.choice(l1)
    spoke2= random.choice(l2)

    solution[spoke1-1] = hubs_chosen[1]
    solution[spoke2-1] = hubs_chosen[0]
    
    solution[hubs_chosen[0]-1] = hubs_chosen[0]
    solution[hubs_chosen[1]-1] = hubs_chosen[1]
    
    return solution



In [ ]:
#NEIGHBOURHOOD STRUCTURE 4:Reallocating a node
def NS4(solution, node):

    hubs = list(set(solution))
    nodes = list(range(1,node+1))
    for hub in hubs:
        nodes.remove(hub)

    spoke = random.choice(nodes)

    hub1 = solution[spoke-1]
    hubs.remove(hub1)
    hub2 = random.choice(hubs)

    solution[spoke-1] = hub2
    return solution
    
    

In [ ]:
#NEIGHBOURHOOD STRUCTURE 5: Swapping the nodes of two hubs 
def NS5(solution,node):

    hubs = list(set(solution))
    hubs_chosen = random.sample(hubs, 2)

    hub_spoke = {}
    for hub in set(solution):
        hub_spoke[hub] = [i for i,val in enumerate(solution) if val==hub]#it takes the idexes of the hubs in the solution
        for spokes in list(hub_spoke.values()):
            hub_spoke[hub] = [index for index in spokes if index != hub-1] #hubs and their spokes' indexes 
    
    for i in hub_spoke[hubs_chosen[0]]:
        solution[i] = hubs_chosen[1]
    for i in hub_spoke[hubs_chosen[1]]:
        solution[i] = hubs_chosen[0]
    

    
        
    return solution
 

In [ ]:
def total_hub_flow(solution, node, flow_matrix):

    hub_spoke = {}
    for hub in set(solution):
        hub_spoke[hub] = [i for i,val in enumerate(solution) if val == hub] #takes the indexes of the hubs in the string
        for spokes in list(hub_spoke.values()):
            hub_spoke[hub] = [index for index in spokes if index != hub-1] #hubs and their spokes' indexes 
    hub_flows = {}
    from_hub_nodes_to_other_nodes = {} #including the parent hub and the other hubs
    for hub in set(solution):
        hub_flows[hub] = [list(flow_matrix.iloc[hub-1])] #from_hub_itself_to_other_nodes
        from_hub_nodes_to_other_nodes[hub] = [list(flow_matrix.iloc[spoke_index]) for spoke_index in hub_spoke[hub]]
        for flow_list in from_hub_nodes_to_other_nodes[hub]:
            hub_flows[hub].append(flow_list)
            
    total_hub_flow = {}
    for hub in set(solution):
        total_hub_flow[hub] = sum(list(itertools.chain.from_iterable(hub_flows[hub])))         
    return total_hub_flow


In [ ]:
#FEASIBILITY CONTROL
def feasibility_check(solution, node, flow_matrix, capacity_matrix):
    for flow in list(total_hub_flow(solution, node, flow_matrix ).values()):
        if flow <= max(capacity_matrix):
            return 1
        else:
            
            return 0
    
    

In [ ]:
#CHOOSE THE CAPACITIES OF THE HUBS
def capacities(solution, node, capacity_matrix, flow_matrix):
    hub_capacities = {}
    for hub in set(solution):
        if total_hub_flow(solution, node, flow_matrix)[hub] <= capacity_matrix[0]:
            
            hub_capacities[hub] = capacity_matrix[0]
            
        elif capacity_matrix[0] < total_hub_flow(solution, node, flow_matrix)[hub] <= capacity_matrix[1]:
            
            hub_capacities[hub] = capacity_matrix[1]
        else:
            hub_capacities[hub] = capacity_matrix[2]
            
    return hub_capacities



In [ ]:
capacities([1, 20, 20, 4, 4, 20, 11, 11, 20, 11, 11, 22, 1, 1, 4, 11, 20, 20, 22, 20, 4, 22, 22, 1, 20], 25, cab_25_capacity, CAB_25_nodes_flow)  


In [ ]:
def total_network_cost(solution, node, flow_matrix, cost_matrix, alpha, capacity_matrix, fixed_cost_matrix):
    total_flow_cost = 0
    for i in range(0, node):
        for j in range(0, node):
            flow_cost = flow_matrix.iloc[i][j]*(cost_matrix.iloc[i][solution[i]-1] + alpha*cost_matrix.iloc[solution[i]-1][solution[j]-1] +cost_matrix.iloc[solution[j]-1][j] )
            total_flow_cost = total_flow_cost + flow_cost
    hub_capacities = list(capacities(solution, node, capacity_matrix, flow_matrix).values())
    capacity_costs = []
    for capacity in hub_capacities:
        if capacity <= capacity_matrix[0]:
            capacity_costs.append(fixed_cost_matrix[0])
            
        elif capacity_matrix[0]< capacity < capacity_matrix[2]:
            capacity_costs.append(fixed_cost_matrix[1])
        else:
            capacity_costs.append(fixed_cost_matrix[2])
    
    total_fixed_costs = sum(capacity_costs)
    
    total_system_cost = total_flow_cost + total_fixed_costs
    return total_system_cost
            

In [ ]:
total_network_cost( [5, 2, 2, 4, 5, 4, 5, 5, 4, 5], 10, CAB_10_nodes_flow, CAB_10_nodes_cost, 0.2, cab_10_capacity, hub_fixed_costs)


# GENETIC ALGORITHM

In [ ]:
#INITIAL POPULATION - randomly allocated hubs and spokes

def initial_population(node, n_hub, flow_matrix, capacity_matrix, n_solution):
    df_random_solutions = pd.DataFrame()
    for i in range(n_solution ):
        solution = random_initial_solution_generator(node,n_hub)
        if feasibility_check(solution, node, flow_matrix, capacity_matrix) == 1 :
            df_random_solutions[i] = solution

    initial_population = [tuple(list(df_random_solutions[i])) for i in range(0,n_solution )]
    in_population = [list(individual) for individual in initial_population]
        
    
    return in_population
          
          

In [ ]:
def least_cost_initial_population(node, n_hub, flow_matrix, cost_matrix, capacity_matrix, n_solution):
    df_solutions = pd.DataFrame()
    for i in range(n_solution):
        solution = random_initial_solution_generator(node,n_hub)
        if feasibility_check(solution, node, flow_matrix, capacity_matrix) == 1 :
            df_solutions[i] = solution

    initial_population = [tuple(list(df_solutions[i])) for i in range(0,n_solution)]
    in_population = [list(individual) for individual in initial_population]
        
    
    return in_population
          

In [ ]:
#EVALUATION OF CHROMOSOME FITNESS

def fitness_evaluation(population, node, flow_matrix, cost_matrix,alpha, capacity_matrix, fixed_cost_matrix):
    solution_cost = []
    for solution in population:
        cost = total_network_cost(solution, node, flow_matrix, cost_matrix, alpha, capacity_matrix, fixed_cost_matrix)
        solution_cost.append((cost, solution))

        
    return solution_cost

In [ ]:
def tournament_selection(fitness_values_and_solutions, k):

    chosen_individuals1 = random.sample(fitness_values_and_solutions, k)
    parent1_fitness = min([individual[0] for individual in chosen_individuals1])
    for individual in chosen_individuals1:
        if individual[0] == parent1_fitness:
            parent1 = individual

    chosen_individuals2 = random.sample(fitness_values_and_solutions, k)
    parent2_fitness = min([individual[0] for individual in chosen_individuals2])
    for individual in chosen_individuals2:
        if individual[0] == parent2_fitness:
            parent2 = individual
            
    parents = [parent[1] for parent in [parent1, parent2] ]
    
    return parents    
    

In [ ]:
def crossover(parents, node):
    random_point = random.choice(range(0, node))
    parent1 = random.choice(parents)
    offspring = [0]*node
    
    for i in range(0, random_point+1):
        offspring.pop(i)
        offspring.insert(i, parent1[i])
        
    for parent in parents:
        if parent != parent1:
            parent2 = parent
    parent2 = parent2
    for i in range(random_point, node):
        offspring.pop(i)
        offspring.insert(i, parent2[i])
    
        
    return offspring
    

In [ ]:
def offspring_repair(offspring, node,  n_hub): #of if len(set(offspring)) > n_hub in the main function
 
    hubs_offspring = []
    
    if len(set(offspring)) == n_hub:
        for hub in set(offspring):
            offspring[hub-1] = hub
    
    elif len(set(offspring)) > n_hub:  
        for i in range(1500) :
            if len(set(offspring)) == n_hub:
                break
            for hub in set(offspring):
                for i in range(0, node):
                    if offspring[i] not in hubs_offspring:
                        hubs_offspring.append(hub)                

            hub_spoke_indexes = {}

            for hub in hubs_offspring:
                hub_spoke_indexes[hub] = [i for i,val in enumerate(offspring) if val == hub] #takes the indexes of the hubs in the string
            hub_to_remove = hubs_offspring.pop(-1)
            
            hubs = list(set(hubs_offspring))
            chosen_hub = random.choice(hubs)
            for index in hub_spoke_indexes[hub_to_remove]:
                offspring.pop(index)
                offspring.insert(index, chosen_hub)
            for hub in hubs:
                offspring.pop(hub-1)
                offspring.insert(hub-1, hub)
            
                
    
    return offspring
    
    
    
    
    
    
    

In [ ]:
def feasibility_offspring(offspring, n_hubs, node, flow_matrix, capacity_matrix):
    if len(set(offspring)) == n_hubs and feasibility_check(offspring, node, flow_matrix, capacity_matrix) == 1: # and offspring != list(parents[0]) and offspring != list(parents[1]):
        return 1
    else:
        return 0
        
    

In [ ]:
def mutation(offspring, node):
    operation_list = ["NS4", "NS3", "NS1"]
    chosen_operation = random.choice(operation_list)
    if chosen_operation == "NS4":
        return NS4(offspring, node)
    elif chosen_operation == "NS3":
        return NS3(offspring,node)
    else:
        return NS1(offspring, node)
    

# GENETIC ALGORITHM IMPLEMENTATION


In [ ]:
def genetic_algorithm(node, n_hub, k, flow_matrix, cost_matrix, capacity_matrix, alpha,  fixed_cost_matrix, crossover_prob, mutation_prob, n_solution):
    start = time.time()
    t = 0
    offsprings = []
    #The initial population
    init_pop = least_cost_initial_population(node, n_hub, flow_matrix, cost_matrix, capacity_matrix, n_solution)
    #Initial population evaluation
    pop_evaluation = fitness_evaluation(init_pop, node, flow_matrix, cost_matrix, alpha, capacity_matrix, fixed_cost_matrix)
    global generations, final_generations, min_values 
    generations = []
    final_generations = []
    counter = 0
    while counter < 2:
        t = t + 1
        #print("t:", t, "number of offspring:", len(offsprings))
        parents = tournament_selection(pop_evaluation, k) #random da olabilir
        
        #parents = random.sample(init_pop, 2)
        rand_crossover = random.uniform(0, 1)
        #print("prob cross:", rand_crossover)
        
        try:
            if rand_crossover <= crossover_prob:
                offspring_crossovered = crossover(parents, node)
                offspring_crossovered_fixed = offspring_repair(offspring_crossovered, node, n_hub)
                evaluation_c = total_network_cost(offspring_crossovered_fixed, node, flow_matrix, cost_matrix, alpha, capacity_matrix, fixed_cost_matrix)
                offspring_crossovered_mutated = mutation(offspring_crossovered_fixed, node) 
                evaluation_cm = total_network_cost(offspring_crossovered_mutated, node, flow_matrix, cost_matrix, alpha, capacity_matrix, fixed_cost_matrix)
                if evaluation_cm < evaluation_c:
                    offsprings.append(offspring_crossovered_mutated)
                else:
                    offsprings.append(offspring_crossovered)
                    
            else:
                chosen_parent = random.choice(parents)
                parent_evaluation = total_network_cost(chosen_parent, node, flow_matrix, cost_matrix, alpha, capacity_matrix, fixed_cost_matrix)
                rand_mutation = random.uniform(0, 1)
                if rand_mutation <= mutation_prob:
                    offspring_mutated = mutation(chosen_parent, node)
                    evaluation_m = total_network_cost(chosen_parent, node, flow_matrix, cost_matrix, alpha, capacity_matrix, fixed_cost_matrix)
                    if evaluation_m < parent_evaluation:
                        offsprings.append(offspring_mutated)
                    else:
                        offsprings.append(chosen_parent)
                else:
                    offsprings.append(chosen_parent)

                generations.append(offsprings)
        except:
            xxx = 0
        #print("offsprings:", offsprings)
        if len(offsprings) == len(init_pop): #if t%n_solution == 0:
            #final_generations.append(init_pop)#*****
            parents=offsprings   #init_pop = offsprings idi öncesinde
            init_pop = offsprings
            offsprings = []
            counter = counter +1
            final_generations.append(parents)     #append(offspring idi)
            parents = []
            
            
            
    min_value_chromosomes = []
    for generation in final_generations:      
        fitness_offsprings = fitness_evaluation(generation, node, flow_matrix, cost_matrix, alpha, capacity_matrix, fixed_cost_matrix)
        evaluation_offsprings = [fitness_offsprings[i][0] for i in range(len(fitness_offsprings))]
        evaluation_offsprings.sort()
        evaluation_offsprings_and_chromosome = []
        for evaluation in evaluation_offsprings:
            for i in range(len(fitness_offsprings)):
                if evaluation == fitness_offsprings[i][0]:
                    evaluation_offsprings_and_chromosome.append((evaluation,fitness_offsprings[i][1]))
                    min_value_chromosomes.append(evaluation_offsprings_and_chromosome[0])
    min_values = []
    for min_value_chromosome in min_value_chromosomes:
        min_values.append(min_value_chromosome[0])
        
    for min_value_chromosome in min_value_chromosomes:
        if min(min_values) == min_value_chromosome[0]:
            end = time.time()
            #print("time elapsed:",end-start)
            #print(min_value_chromosome)
            return min_value_chromosome,end-start

           
        
    
    
    
    
    
    
    
    
    
    

In [ ]:
print("CAB 10")

hub_numbers = [3, 5]
alphas = [0.2, 0.4, 0.8]

for n_hub in hub_numbers:
    for alpha in alphas:
        costs = []
        print("n_hub:", n_hub, "alpha:", alpha)
        for i in range(10):
            a = genetic_algorithm(10, n_hub, 4 ,CAB_10_nodes_flow, CAB_10_nodes_cost, cab_10_capacity, alpha, hub_fixed_costs, 0.85, 0.90, 150)
            print(a)
            costs.append(a[0][0])
        print("Average cost:", sum(costs)/10)
        print(" ")
        costs = []

        

In [ ]:
print("CAB 25")

hub_numbers = [3, 5]
alphas = [0.2, 0.4, 0.8]

for n_hub in hub_numbers:
    for alpha in alphas:
        costs = []
        print("n_hub:", n_hub, "alpha:", alpha)
        for i in range(10):
            a = genetic_algorithm(25, n_hub, 4 ,CAB_25_nodes_flow, CAB_25_nodes_cost, cab_25_capacity, alpha, hub_fixed_costs, 0.85, 0.90, 100)
            print(a)
            costs.append(a[0][0])
        print("Average cost:", sum(costs)/10)
        print(" ")
        costs = []


In [ ]:
print("TR 25")

hub_numbers = [3, 5]
alphas = [0.2, 0.4, 0.8]

for n_hub in hub_numbers:
    for alpha in alphas:
        costs = []
        print("n_hub:", n_hub, "alpha:", alpha)
        for i in range(10):
            a = genetic_algorithm(25, n_hub, 4 ,TR_25_nodes_flow, TR_25_nodes_cost, tr_25_capacity, alpha, hub_fixed_costs, 0.85, 0.90, 100)
            print(a)
            costs.append(a[0][0])
        print("Average cost:", sum(costs)/10)
        print(" ")
        costs = []


In [ ]:
print("TR 55")

hub_numbers = [3, 5]
alphas = [0.2, 0.4, 0.8]

for n_hub in hub_numbers:
    for alpha in alphas:
        costs = []
        print("n_hub:", n_hub, "alpha:", alpha)
        for i in range(10):
            a = genetic_algorithm(25, n_hub, 4 ,TR_55_nodes_flow, TR_55_nodes_cost, tr_55_capacity, alpha, hub_fixed_costs, 0.85, 0.90, 100)
            print(a)
            costs.append(a[0][0])
        print("Average cost:", sum(costs)/10)
        print(" ")
        costs = []


# VNS

In [ ]:
#VARIABLE NEIGHBOURHOOD DESCENT

def VND( node, n_hub, flow_matrix, cost_matrix, alpha, capacity_matrix, fixed_cost_matrix, n_iterations):
    start = time.time()
    solution = least_cost_random_hub_based_solution_generator(node, n_hub, cost_matrix)
    solution_cost = total_network_cost(solution, node, flow_matrix, cost_matrix, alpha, capacity_matrix, fixed_cost_matrix )
    counter = 0
    k = 1
    while counter < n_iterations:
        sol1 = solution.copy()
        structures= [NS1,NS2,NS3,NS4,NS5]
        random.shuffle(structures)
        
        neighbours = []
        for st in structures:
            n = st(sol1, node)
            neighbour = n.copy()
            neighbours.append(neighbour)

        neighbour_costs = [total_network_cost(neighbour, node, flow_matrix, cost_matrix, alpha, capacity_matrix, fixed_cost_matrix ) for neighbour in neighbours]
        
        if min(neighbour_costs) < solution_cost:
            best_neighbour = [neighbour for neighbour in neighbours if min(neighbour_costs) == total_network_cost(neighbour, node, flow_matrix, cost_matrix, alpha, capacity_matrix, fixed_cost_matrix )][0]
            best_neighbour_cost = total_network_cost(best_neighbour, node, flow_matrix, cost_matrix, alpha, capacity_matrix, fixed_cost_matrix )
            solution = best_neighbour
            solution_cost = best_neighbour_cost
            counter = counter +1 
            
        else:

            counter = counter + 1
            
    end = time.time()  
    
    #print(solution_cost, solution, end-start)
    return (solution_cost, solution, end-start)
                

In [ ]:
print("CAB 10")

hub_numbers = [3, 5]
alphas = [0.2, 0.4, 0.8]

for n_hub in hub_numbers:
    for alpha in alphas:
        costs = []
        print("n_hub:", n_hub, "alpha:", alpha)
        for i in range(10):
            a = VND(10, n_hub, CAB_10_nodes_flow, CAB_10_nodes_cost, alpha, cab_10_capacity, hub_fixed_costs, 10)
            print(a)
            costs.append(a[0])
        print("Average cost:", sum(costs)/10)
        print(" ")
        costs = []

        
  
        

In [ ]:
print("CAB 25")

hub_numbers = [3, 5]
alphas = [0.2, 0.4, 0.8]

for n_hub in hub_numbers:
    for alpha in alphas:
        costs = []
        print("n_hub:", n_hub, "alpha:", alpha)
        for i in range(10):
            a = VND(25, n_hub, CAB_25_nodes_flow, CAB_25_nodes_cost, alpha, cab_25_capacity, hub_fixed_costs, 20)
            print(a)
            costs.append(a[0])
        print("Average cost:", sum(costs)/10)
        print(" ")
        costs = []
  
        

In [ ]:
print("TR 25")

hub_numbers = [3, 5]
alphas = [0.2, 0.4, 0.8]

for n_hub in hub_numbers:
    for alpha in alphas:
        costs = []
        print("n_hub:", n_hub, "alpha:", alpha)
        for i in range(10):
            a = VND(25, n_hub, TR_25_nodes_flow, TR_25_nodes_cost, alpha, tr_25_capacity, hub_fixed_costs, 20)
            print(a)
            costs.append(a[0])
        print("Average cost:", sum(costs)/10)
        print(" ")
        costs = []
  
  

In [ ]:
print("TR 55")

hub_numbers = [3, 5]
alphas = [0.2, 0.4, 0.8]

for n_hub in hub_numbers:
    for alpha in alphas:
        costs = []
        print("n_hub:", n_hub, "alpha:", alpha)
        for i in range(10):
            a = VND(55, n_hub, TR_55_nodes_flow, TR_55_nodes_cost, alpha, tr_55_capacity, hub_fixed_costs, 20)
            print(a)
            costs.append(a[0])
        print("Average cost:", sum(costs)/10)
        print(" ")
        costs = []
  
  

In [ ]:
#REDUCED VARIABLE NEIGHBOURHOOD SEARCH

def RVND( node, n_hub, flow_matrix, cost_matrix, alpha, capacity_matrix, fixed_cost_matrix, n_iterations):
    start = time.time()
    solution = least_cost_random_hub_based_solution_generator(node, n_hub, cost_matrix)
    solution_cost = total_network_cost(solution, node, flow_matrix, cost_matrix, alpha, capacity_matrix, fixed_cost_matrix )
    counter = 0
    k = 1
    while counter < n_iterations:
        sol1 = solution.copy()
        structures= [NS1,NS2,NS3,NS4,NS5]
        random.shuffle(structures)
        chosen = random.choice(structures)
        n = chosen(sol1, node)
        neighbour = n.copy()

        neighbour_cost = total_network_cost(neighbour, node, flow_matrix, cost_matrix, alpha, capacity_matrix, fixed_cost_matrix ) 
        
        if neighbour_cost< solution_cost:
            solution = neighbour
            solution_cost = neighbour_cost
            counter = counter +1 
        else:
            counter = counter + 1
            
    end = time.time()  
    
    
    return (solution_cost, solution, end-start)
                

In [ ]:
print("CAB 10")

hub_numbers = [3, 5]
alphas = [0.2, 0.4, 0.8]

for n_hub in hub_numbers:
    for alpha in alphas:
        costs = []
        print("n_hub:", n_hub, "alpha:", alpha)
        for i in range(10):
            a = RVND(10, n_hub, CAB_10_nodes_flow, CAB_10_nodes_cost, alpha, cab_10_capacity, hub_fixed_costs, 10)
            print(a)
            costs.append(a[0])
        print("Average cost:", sum(costs)/10)
        print(" ")
        costs = []
  

In [ ]:
print("CAB 25")

hub_numbers = [3, 5]
alphas = [0.2, 0.4, 0.8]

for n_hub in hub_numbers:
    for alpha in alphas:
        costs = []
        print("n_hub:", n_hub, "alpha:", alpha)
        for i in range(10):
            a = RVND(25, n_hub, CAB_25_nodes_flow, CAB_25_nodes_cost, alpha, cab_25_capacity, hub_fixed_costs, 20)
            print(a)
            costs.append(a[0])
        print("Average cost:", sum(costs)/10)
        print(" ")
        costs = []
  

In [ ]:
print("TR 25")

hub_numbers = [3, 5]
alphas = [0.2, 0.4, 0.8]

for n_hub in hub_numbers:
    for alpha in alphas:
        costs = []
        print("n_hub:", n_hub, "alpha:", alpha)
        for i in range(10):
            a = RVND(25, n_hub, TR_25_nodes_flow, TR_25_nodes_cost, alpha, tr_25_capacity, hub_fixed_costs,20)
            print(a)
            costs.append(a[0])
        print("Average cost:", sum(costs)/10)
        print(" ")
        costs = []
  

In [ ]:
print("TR 55")

hub_numbers = [3, 5]
alphas = [0.2, 0.4, 0.8]

for n_hub in hub_numbers:
    for alpha in alphas:
        costs = []
        print("n_hub:", n_hub, "alpha:", alpha)
        for i in range(10):
            a = RVND(55, n_hub, TR_55_nodes_flow, TR_55_nodes_cost, alpha, tr_55_capacity, hub_fixed_costs, 20)
            print(a)
            costs.append(a[0])
        print("Average cost:", sum(costs)/10)
        print(" ")
        costs = []
  

In [ ]:
#REVISED REDUCED VARIABLE NEIGHBOURHOOD SEARCH

def RRVND( node, n_hub, flow_matrix, cost_matrix, alpha, capacity_matrix, fixed_cost_matrix, n_iterations):
    start = time.time()
    solution = least_cost_random_hub_based_solution_generator(node, n_hub, cost_matrix)
    solution_cost = total_network_cost(solution, node, flow_matrix, cost_matrix, alpha, capacity_matrix, fixed_cost_matrix )
    counter = 0
    k = 1
    while counter < n_iterations:

        
        sol1 = solution.copy()
        
   
        structures= [NS1,NS2,NS3,NS4,NS5]
        random.shuffle(structures)
        chosen = random.choice(structures)
        
        neighbours = []
        #for ns in structures:
        for i in range(5):
            n = chosen(sol1, node)   #ns(sol1, node)
            neighbour = n.copy()
            neighbours.append(neighbour)
        

        neighbour_costs = [total_network_cost(neighbour, node, flow_matrix, cost_matrix, alpha, capacity_matrix, fixed_cost_matrix ) for neighbour in neighbours]
        
        if min(neighbour_costs) < solution_cost:
            best_neighbour = [neighbour for neighbour in neighbours if min(neighbour_costs) == total_network_cost(neighbour, node, flow_matrix, cost_matrix, alpha, capacity_matrix, fixed_cost_matrix )][0]
            best_neighbour_cost = total_network_cost(best_neighbour, node, flow_matrix, cost_matrix, alpha, capacity_matrix, fixed_cost_matrix )
            solution = best_neighbour
            solution_cost = best_neighbour_cost
            counter = counter +1 
            
        else:
            counter = counter + 1
            
    end = time.time()  
    #print(solution_cost, solution, end-start)
    return (solution_cost, solution, end-start)
                
    
    

In [ ]:
print("CAB 10") # 10 iterations

hub_numbers = [3, 5]
alphas = [0.2, 0.4, 0.8]

for n_hub in hub_numbers:
    for alpha in alphas:
        costs = []
        print("n_hub:", n_hub, "alpha:", alpha)
        for i in range(10):
            a = RRVND(10, n_hub, CAB_10_nodes_flow, CAB_10_nodes_cost, alpha, cab_10_capacity, hub_fixed_costs, 10)
            print(a)
            costs.append(a[0])
        print("Average cost:", sum(costs)/10)
        print(" ")
        costs = []
  

In [ ]:
print("CAB 25")

hub_numbers = [3, 5]
alphas = [0.2, 0.4, 0.8]

for n_hub in hub_numbers:
    for alpha in alphas:
        costs = []
        print("n_hub:", n_hub, "alpha:", alpha)
        for i in range(10):
            a = RRVND(25, n_hub, CAB_25_nodes_flow, CAB_25_nodes_cost, alpha, cab_25_capacity, hub_fixed_costs, 20)
            print(a)
            costs.append(a[0])
        print("Average cost:", sum(costs)/10)
        print(" ")
        costs = []
  

In [ ]:
print("TR 25")

n_hubs = [3, 5]
alphas = [0.2, 0.4, 0.8]

hub_numbers = [3, 5]
alphas = [0.2, 0.4, 0.8]

for n_hub in hub_numbers:
    for alpha in alphas:
        costs = []
        print("n_hub:", n_hub, "alpha:", alpha)
        for i in range(10):
            a = RRVND(25, n_hub, TR_25_nodes_flow, TR_25_nodes_cost, alpha, tr_25_capacity, hub_fixed_costs, 20)
            print(a)
            costs.append(a[0])
        print("Average cost:", sum(costs)/10)
        print(" ")
        costs = []
  

In [ ]:
print("TR 55")

hub_numbers = [3, 5]
alphas = [0.2, 0.4, 0.8]

for n_hub in hub_numbers:
    for alpha in alphas:
        costs = []
        print("n_hub:", n_hub, "alpha:", alpha)
        for i in range(10):
            a = RRVND(55, n_hub, TR_55_nodes_flow, TR_55_nodes_cost, alpha, tr_55_capacity, hub_fixed_costs, 20)
            print(a)
            costs.append(a[0])
        print("Average cost:", sum(costs)/10)
        print(" ")
        costs = []
  